<a href="https://colab.research.google.com/github/adalves-ufabc/2022.Q2-PLN/blob/main/2022_Q2_PLN_Notebook_25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Processamento de Linguagem Natural [2022.Q2]**
Prof. Alexandre Donizeti Alves

### **Sumarização Automática de Textos**

***Part-of-Speech tagging*** (*POS-tagging*) é um dos principais componentes de quase todas as análises de PLN. A tarefa de etiquetagem implica simplesmente em rotular palavras com sua característica de linguagem (i.e., substantivo, verbo, adjetivo, advérbio etc.). Já o **Reconhecimento de Entidades Nomeadas** permite identificar entidades (por exemplo, nomes de pessoas, lugares, empresas etc.) em um documento. 

>
Neste exemplo, vamos usar ***POS-tagging*** e **Reconhecimento de Entidades Nomeadas** para fazer **Summarização Automática de Textos**.


### **Sumarização** 

É um processo de geração automática de um resumo associado a um objeto (e.g., texto, artigo, história). A sumarização permite não somente entender a estrutura de uma frase, mas também a estrutura do texto inteiro. A sumarização permite evidenciar o tema tratado. Existem diversas técnicas para fazer a sumarização, mas neste exemplo, iremos usar uma heurística simples.

>
Iremos assumir que, normalmente, uma frase que tem mais **entidades** e **substantivos** tem maior importância que outras sentenças. Usando essa lógica vamos ver um exemplo "simples" para cálculo de pontuação de importância. Podemos considerar um limiar para selecionar apenas as frases mais importantes.

>
Vamos ver um exemplo dessa heurística criando sumários para notícias:

In [1]:
!pip install nlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import nlp
import nltk

nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
from datasets import load_dataset

multi_news = load_dataset("multi_news")

  0%|          | 0/3 [00:00<?, ?it/s]

Essa base de dados contém um conjunto de notícias e sumários escritos por humanos, usada para treinar ferramentas de sumarização automática. A base está dividida entre treino, validação e teste. Como não vamos usar aprendizado de máquina, podemos escolher qualquer uma. 

In [7]:
multi_news

DatasetDict({
    train: Dataset({
        features: ['document', 'summary'],
        num_rows: 44972
    })
    validation: Dataset({
        features: ['document', 'summary'],
        num_rows: 5622
    })
    test: Dataset({
        features: ['document', 'summary'],
        num_rows: 5622
    })
})

Vamos criar um método que separa um texto em sentenças. Cada sentença é então dividida em tokens, e usamos um *POS-Tagger* e um Reconhecedor de Entidades Nomeadas para encontrar substantivos e entidades nomeadas (a lista de entidades nomeadas reconhecidas é dado pela base '`words`' do `nltk`. A seguir, o método calcula a porcentagem de tokens que são nomes ou entidades nomeadas, e gera um ranking baseado nessa porcentagem. O método retorna uma lista de sentenças, ordenadas por essa pontuação (*score*).

In [8]:
def summary(content):
    ranking = []
    
    # quebra o documento em sentencas
    for (i, sentence) in enumerate(nltk.sent_tokenize(content)):
        
        # quebra a sentenca em palavras
        tokens         = nltk.wordpunct_tokenize(sentence)
        tokens = [token for token in tokens if token.isalpha()]
        
        # faz o pos-tagging
        tagged         = nltk.pos_tag(tokens)
        
        # encontra as entidades nomeadas
        named_entities = nltk.ne_chunk(tagged, binary=True)
        nTokens        = len(tokens)

        #print (f"\n{i} : >>{sentence}<<\n")

        # contagem dos substantivos: NN e NNP
        nNouns = 0
        for (word, pos) in tagged:
            if pos in ["NN", "NNP"]:
                nNouns += 1
                #print (word, pos)

        # contagem de entidades nomeadas
        nNes = 0
        for ne in named_entities:
            if hasattr(ne, 'label') and ne.label() == "NE":
                nNes += 1
                #print (ne)

        # calculo do score ingenuo
        # soma dos nomes e entidades nomeadas
        try:
            score = (nNes + nNouns)/float(nTokens)
        except:
            score = 0
        ranking.append( {'id':i, 'score':score, 'nNes':nNes, 'nNouns':nNouns, 'nTokens':nTokens, 'sentence':sentence} )
    
    # retorna as sentencas, ordenadas pelo ranking
    return(sorted( ranking, key=lambda x: x['score'], reverse=True))


Podemos usar esse método para criar um sumário baseado na heurística, pegando a primeira sentença que tem proporcionalmente mais substativos e entidades nomeadas:

Vamos aplicar o  método a uma das notícias do documento:

In [20]:
multi_news['train']['document'][1]

'LOS ANGELES (AP) — In her first interview since the NBA banned her estranged husband, Shelly Sterling says she will fight to keep her share of the Los Angeles Clippers and plans one day to divorce Donald Sterling. \n \n (Click Prev or Next to continue viewing images.) \n \n ADVERTISEMENT (Click Prev or Next to continue viewing images.) \n \n Los Angeles Clippers co-owner Shelly Sterling, below, watches the Clippers play the Oklahoma City Thunder along with her attorney, Pierce O\'Donnell, in the first half of Game 3 of the Western Conference... (Associated Press) \n \n Shelly Sterling spoke to Barbara Walters, and ABC News posted a short story with excerpts from the conversation Sunday. \n \n NBA Commissioner Adam Silver has banned Donald Sterling for making racist comments and urged owners to force Sterling to sell the team. Silver added that no decisions had been made about the rest of Sterling\'s family. \n \n According to ABC\'s story, Shelly Sterling told Walters: "I will fight t

In [21]:
sents = summary(multi_news['train']['document'][1])

E imprimir a primeira sentença, que tomamos como sumário:

In [22]:
print(sents[0]['sentence'])

Sherry Sterling's lawyer, Pierce O'Donnell, disputed the league's reading of its constitution.


Podemos comparar esse resultado com o sumário do documento, escrito por humanos.

In [23]:
print(multi_news['train']['summary'][1])

– Shelly Sterling plans "eventually" to divorce her estranged husband Donald, she tells Barbara Walters at ABC News. As for her stake in the Los Angeles Clippers, she plans to keep it, the AP notes. Sterling says she would "absolutely" fight any NBA decision to force her to sell the team. The team is her "legacy" to her family, she says. "To be honest with you, I'm wondering if a wife of one of the owners … said those racial slurs, would they oust the husband? Or would they leave the husband in?"


**Referência**:

Este notebook foi criado pelo prof. **Ronaldo Prati** [UFABC], tendo sido adaptado por mim.